**Discriminator**

In [1]:
import torch
import torch.nn as nn

In [2]:
from torch.nn.modules.activation import LeakyReLU
class CNNBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride):
    super().__init__()
    self.conv= nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 4, stride, bias=False, padding_mode="reflect"),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2),
    )
  def forward(self,x):
    return self.conv(x)
#x,y <- concatenate these along the channels
class Discriminator(nn.Module):
  def __init__(self, in_channels=3, features=[64, 128, 256, 512]): #256->30x30
      super().__init__()
      self.initial = nn.Sequential(
          nn.Conv2d(in_channels*2, features[0], kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
          nn.LeakyReLU(0.2),
      )
      layers = []
      in_channels=features[0]
      for feature in features[1:]:
        layers.append(
            CNNBlock(in_channels, feature, stride=1 if feature==features[-1] else 2),
        )
        in_channels = feature
      layers.append(
          nn.Conv2d(in_channels,1,kernel_size=4,stride=1, padding=1, padding_mode="reflect")
      )
      self.model = nn.Sequential(*layers)
  def forward(self, x, y):
    x = torch.cat([x,y], dim=1)
    x = self.initial(x)
    return self.model(x)

def test():
  x = torch.randn([1,3,286,286])
  y = torch.randn([1,3,286,286])
  model = Discriminator()
  preds = model(x,y)
  return preds.shape

if __name__ == "__main__":
  print(test())

torch.Size([1, 1, 30, 30])


**Generator**

In [3]:
from torch.nn.modules.batchnorm import BatchNorm2d
class Block(nn.Module):
  def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
      super().__init__()
      self.conv = nn.Sequential(
          nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
          if down
          else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
          nn.BatchNorm2d(out_channels),
          nn.ReLU() if act=="relu" else nn.LeakyReLU(0.2),
      )
      self.use_dropout = use_dropout
      self.dropout = nn.Dropout(0.5)
  def forward(self, x):
    x = self.conv(x)
    return self.dropout(x) if self.use_dropout else x

class Generator(nn.Module):
  def __init__(self, in_channels=3, features=64):
      super().__init__()
      self.initial_down = nn.Sequential(
          nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
          nn.LeakyReLU(0.2),
      )#128
      self.down1 = Block(features, features*2, down=True, use_dropout=False)#64
      self.down2 = Block(features*2, features*4, down=True, use_dropout=False)#32
      self.down3 = Block(features*4, features*8, down=True, use_dropout=False)#16
      self.down4 = Block(features*8, features*8, down=True, use_dropout=False)#8
      self.down5 = Block(features*8, features*8, down=True, use_dropout=False)#4
      self.down6 = Block(features*8, features*8, down=True, use_dropout=False)#2
      self.bottleneck = nn.Sequential(
          nn.Conv2d(features*8, features*8, 4, 2, 1, padding_mode="reflect"),
          nn.ReLU(),
      )#1
      self.up1 = Block(features*8, features*8, down=False, act="relu", use_dropout=True)
      self.up2 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=True)
      self.up3 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=True)
      self.up4 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=False)
      self.up5 = Block(features*8*2, features*4, down=False, act="relu", use_dropout=False)
      self.up6 = Block(features*4*2, features*2, down=False, act="relu", use_dropout=False)
      self.up7 = Block(features*2*2, features, down=False, act="relu", use_dropout=False)

      self.final_up = nn.Sequential(
          nn.ConvTranspose2d(features*2, in_channels, 4, 2, 1),
          nn.Tanh(),
      )
  def forward(self,x):
    d1 = self.initial_down(x)
    d2 = self.down1(d1)
    d3 = self.down2(d2)
    d4 = self.down3(d3)
    d5 = self.down4(d4)
    d6 = self.down5(d5)
    d7 = self.down6(d6)
    d8 = self.bottleneck(d7)

    u1 = self.up1(d8)
    u2 = self.up2(torch.cat([u1,d7], dim=1))
    u3 = self.up3(torch.cat([u2,d6], dim=1))
    u4 = self.up4(torch.cat([u3,d5], dim=1))
    u5 = self.up5(torch.cat([u4,d4], dim=1))
    u6 = self.up6(torch.cat([u5,d3], dim=1))
    u7 = self.up7(torch.cat([u6,d2], dim=1))
    return self.final_up(torch.cat([u7,d1], dim=1))

def test_gen():
  x = torch.randn([1,3,256,256])
  model = Generator(in_channels=3, features=64)
  preds = model(x)
  return preds.shape

if __name__ == "__main__":
  print(test_gen())


torch.Size([1, 3, 256, 256])


**config**

In [4]:
!pip install albumentations==1.0.3

     |████████████████████████████████| 98 kB 3.2 MB/s 
     |████████████████████████████████| 47.8 MB 1.3 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [5]:
!pip install opencv-python-headless==4.1.2.30

     |████████████████████████████████| 21.8 MB 4.0 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.64
    Uninstalling opencv-python-headless-4.5.5.64:
      Successfully uninstalled opencv-python-headless-4.5.5.64


In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [7]:
!nvidia-smi

Wed May  4 02:47:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "../content/drive/MyDrive/AI/maps/maps/train"
VAL_DIR = "../content/drive/MyDrive/AI/maps/maps/val"
LEARNING_RATE = 2e-4
BATCH_SIZE = 16
NUM_WORKERS = 1
IMAGE_SIZE = 256
CHANNELS_IMG = 3
L1_LAMBDA = 100
NUM_EPOCHS = 500
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_DISC = "disc.pth.tar"
CHECKPOINT_GEN = "gen.pth.tar"

both_transform = A.Compose( 
    [A.Resize(width=256, height=256),
    A.HorizontalFlip(p=0.5),],
    additional_targets={"image0":"image"},)
transform_only_input = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.ColorJitter(p=0.2),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)

transform_only_mask = A.Compose(
    [
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)


**dataset**

In [15]:
from PIL import Image
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
import sys
class MapDataset():
  def __init__(self, root_dir):
    self.root_dir = root_dir
    self.list_files =os.listdir(self.root_dir)
    print(self.list_files)
  def __len__(self):
    return(len(self.list_files))
  def __getitem__(self, index):
    img_file = self.list_files[index]
    img_path = os.path.join(self.root_dir, img_file)
    image = np.array(Image.open(img_path))
    input_image = image[:,:600,:] #channel, width, height
    target_image = image[:,600:,:]
    augmentations = both_transform(image=input_image, image0=target_image)
    input_image, target_image=augmentations["image"], augmentations["image0"]
    input_image = transform_only_input(image=input_image)["image"]
    target_image = transform_only_mask(image=target_image)["image"]
    return input_image, target_image


if __name__ == "__main__":
    dataset = MapDataset("../content/drive/MyDrive/AI/maps/maps/train/")
    loader = DataLoader(dataset, batch_size=5)
    for x,y in loader:
      print(x.shape)
      save_image(x, "x.png")
      save_image(y, "y.png")
      sys.exit()

['1066.jpg', '1080.jpg', '1095.jpg', '149.jpg', '1032.jpg', '168.jpg', '143.jpg', '1058.jpg', '139.jpg', '1086.jpg', '1012.jpg', '1079.jpg', '1006.jpg', '11.jpg', '117.jpg', '1085.jpg', '119.jpg', '1048.jpg', '1040.jpg', '137.jpg', '1017.jpg', '1090.jpg', '106.jpg', '1023.jpg', '1077.jpg', '1074.jpg', '1073.jpg', '153.jpg', '1071.jpg', '1061.jpg', '108.jpg', '157.jpg', '1087.jpg', '1091.jpg', '112.jpg', '123.jpg', '128.jpg', '10.jpg', '1045.jpg', '1037.jpg', '1069.jpg', '1083.jpg', '1094.jpg', '1036.jpg', '1.jpg', '1056.jpg', '166.jpg', '113.jpg', '161.jpg', '141.jpg', '145.jpg', '150.jpg', '1000.jpg', '15.jpg', '1075.jpg', '1038.jpg', '1020.jpg', '100.jpg', '1051.jpg', '1049.jpg', '1053.jpg', '1039.jpg', '12.jpg', '167.jpg', '110.jpg', '105.jpg', '124.jpg', '1005.jpg', '1025.jpg', '133.jpg', '1008.jpg', '1022.jpg', '154.jpg', '104.jpg', '101.jpg', '1076.jpg', '121.jpg', '107.jpg', '132.jpg', '158.jpg', '204.jpg', '317.jpg', '327.jpg', '246.jpg', '173.jpg', '29.jpg', '297.jpg', '32.jpg

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**util**

In [16]:
import torch
from torchvision.utils import save_image
from google.colab import files




def save_some_examples(gen, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization#
        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        if epoch == 1:
          save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")

    gen.train()


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


In [17]:
# example of calculating the frechet inception distance in Keras for cifar10
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

**train**

In [18]:
import torch.optim as optim
from tqdm import tqdm

def train_fn(disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, g_scaler, d_scaler):
  loop = tqdm(loader, leave=True)
  
  for idx, (x, y) in enumerate(loop):
    x = x.to(DEVICE)
    y = y.to(DEVICE)

    #Train Discriminator
    with torch.cuda.amp.autocast():
      y_fake = gen(x)
      D_real = disc(x, y)
      D_real_loss = bce(D_real, torch.ones_like(D_real))
      D_fake = disc(x, y_fake.detach())
      D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
      D_loss = (D_real_loss + D_fake_loss)/2
    disc.zero_grad()
    d_scaler.scale(D_loss).backward()
    d_scaler.step(opt_disc)
    d_scaler.update()

    #Train Generator
    with torch.cuda.amp.autocast():
      D_fake=disc(x, y_fake)
      G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
      L1 = l1_loss(y_fake, y)*L1_LAMBDA
      G_loss = G_fake_loss + L1
    gen.zero_grad()
    g_scaler.scale(G_loss).backward()
    g_scaler.step(opt_gen)
    g_scaler.update()    

    if idx % 10 == 0:
      loop.set_postfix(
        D_real=torch.sigmoid(D_real).mean().item(),
        D_fake=torch.sigmoid(D_fake).mean().item(),
      )
    if idx % 50 == 0:
      save_image(y_fake*0.5+0.5, f"../content/drive/MyDrive/AI/maps/evaluation2/fake_y_{idx}.png")
      save_image(y*0.5+0.5, f"../content/drive/MyDrive/AI/maps/evaluation2/real_y_{idx}.png")
      save_image(x*0.5+0.5, f"../content/drive/MyDrive/AI/maps/evaluation2/x_{idx}.png")
def train():
  disc = Discriminator(in_channels=3).to(DEVICE)
  gen = Generator(in_channels=3, features=64).to(DEVICE)
  opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5,0.999),)
  opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5,0.999))
  BCE = nn.BCEWithLogitsLoss()
  L1_LOSS = nn.L1Loss()

  if LOAD_MODEL:
    load_checkpoint(
        CHECKPOINT_GEN, gen, opt_gen, LEARNING_RATE
    )

    load_checkpoint(
        CHECKPOINT_DISC, disc, opt_disc, LEARNING_RATE
    )
  train_dataset = MapDataset(root_dir=TRAIN_DIR)
  train_loader = DataLoader(
      train_dataset,
      batch_size=BATCH_SIZE,
      shuffle=True,
      num_workers=NUM_WORKERS,
  )
  g_scaler = torch.cuda.amp.GradScaler()
  d_scaler = torch.cuda.amp.GradScaler()
  val_dataset = MapDataset(root_dir=VAL_DIR)
  val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
  

  for epoch in range(NUM_EPOCHS):
    train_fn(
        disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler,
    )
    if SAVE_MODEL and epoch % 5 == 0:
      save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN)
      save_checkpoint(disc, opt_disc, filename=CHECKPOINT_DISC)

   # save_some_examples(gen, val_loader, epoch, folder="../content/drive/MyDrive/AI/maps/evaluation")


if __name__=="__main__":
  train()

['1066.jpg', '1080.jpg', '1095.jpg', '149.jpg', '1032.jpg', '168.jpg', '143.jpg', '1058.jpg', '139.jpg', '1086.jpg', '1012.jpg', '1079.jpg', '1006.jpg', '11.jpg', '117.jpg', '1085.jpg', '119.jpg', '1048.jpg', '1040.jpg', '137.jpg', '1017.jpg', '1090.jpg', '106.jpg', '1023.jpg', '1077.jpg', '1074.jpg', '1073.jpg', '153.jpg', '1071.jpg', '1061.jpg', '108.jpg', '157.jpg', '1087.jpg', '1091.jpg', '112.jpg', '123.jpg', '128.jpg', '10.jpg', '1045.jpg', '1037.jpg', '1069.jpg', '1083.jpg', '1094.jpg', '1036.jpg', '1.jpg', '1056.jpg', '166.jpg', '113.jpg', '161.jpg', '141.jpg', '145.jpg', '150.jpg', '1000.jpg', '15.jpg', '1075.jpg', '1038.jpg', '1020.jpg', '100.jpg', '1051.jpg', '1049.jpg', '1053.jpg', '1039.jpg', '12.jpg', '167.jpg', '110.jpg', '105.jpg', '124.jpg', '1005.jpg', '1025.jpg', '133.jpg', '1008.jpg', '1022.jpg', '154.jpg', '104.jpg', '101.jpg', '1076.jpg', '121.jpg', '107.jpg', '132.jpg', '158.jpg', '204.jpg', '317.jpg', '327.jpg', '246.jpg', '173.jpg', '29.jpg', '297.jpg', '32.jpg

100%|██████████| 69/69 [00:35<00:00,  1.93it/s, D_fake=0.153, D_real=0.769]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 69/69 [00:24<00:00,  2.77it/s, D_fake=0.167, D_real=0.834]


=> Saving checkpoint
=> Saving checkpoint


...

100%|██████████| 69/69 [00:25<00:00,  2.76it/s, D_fake=0.0923, D_real=0.813]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 69/69 [00:25<00:00,  2.75it/s, D_fake=0.0148, D_real=0.756]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 69/69 [00:25<00:00,  2.75it/s, D_fake=0.0534, D_real=0.881]


In [19]:
path_fake_y = "../content/drive/MyDrive/AI/maps/evaluation2/fake_y_0.png"
path_real_y = "../content/drive/MyDrive/AI/maps/evaluation2/real_y_0.png"   
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
images1 = np.array(Image.open(path_fake_y)).astype('float32')
images2 = np.array(Image.open(path_real_y)).astype('float32')
# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
# calculate fid
fid = calculate_fid(model, images1, images2)

print('FID: %.3f' % fid)

87924736/87910968 [==============================] - 1s 0us/step
FID: 32.748


In [ ]:
download_files = os.listdir("../content/evaluation")
print(download_files)
for i in download_files:
  files.download("../content/evaluation/" + i)  


['input_143.png', 'input_405.png', 'input_113.png', 'y_gen_342.png', 'y_gen_317.png', 'input_314.png', 'y_gen_404.png', 'y_gen_369.png', 'input_404.png', 'y_gen_318.png', 'y_gen_480.png', 'y_gen_300.png', 'y_gen_199.png', 'y_gen_418.png', 'input_329.png', 'input_20.png', 'y_gen_489.png', 'y_gen_216.png', 'input_274.png', 'y_gen_113.png', 'input_263.png', 'input_135.png', 'y_gen_233.png', 'y_gen_391.png', 'y_gen_379.png', 'input_494.png', 'y_gen_403.png', 'input_322.png', 'y_gen_337.png', 'y_gen_10.png', 'y_gen_147.png', 'input_174.png', 'y_gen_335.png', 'input_154.png', 'input_128.png', 'y_gen_486.png', 'y_gen_255.png', 'input_153.png', 'y_gen_311.png', 'input_220.png', 'y_gen_4.png', 'y_gen_389.png', 'y_gen_140.png', 'input_390.png', 'input_151.png', 'input_298.png', 'input_142.png', 'input_297.png', 'input_373.png', 'input_103.png', 'y_gen_26.png', 'y_gen_356.png', 'input_147.png', 'y_gen_149.png', 'y_gen_201.png', 'input_41.png', 'y_gen_277.png', 'input_435.png', 'input_30.png', 'in

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>